# [Module 1] 데이타 준비 (Prepare data file) 

* 이 노트북에서는 입력 데이타를 Forecast에 입력이 될 포맷으로 학습 (Train) 과 검증셋(Validation)으로 나누는 작업을 합니다. **자세한 과정을 알고 싶으시면 0.1.Option-Prepare-Data-From-Scratch.ipynb 노트북 을 실행 해주세요.** 

    - 데이타는 아래 데이타 소스에서 train.csv 파일을 다운로드 받아서 data 폴더에 train.csv로 저장한 것을 사용 합니다.
    

* 이 노트북은 약 3분 정도 소요 됩니다. 
    
---
- Data Source: Store Item Demand Forecasting Challenge,  https://www.kaggle.com/c/demand-

In [1]:
import os
import pandas as pd
# import json
# import time
import numpy as np

## 데이터 포맷 변경
- 실제 Raw 데이터를 Forecast에 들어갈 데이타 포맷으로 바꾸는 작업을 합니다.

In [2]:
data_dir = 'data'
data_file_name = 'train.csv'
df = pd.read_csv(os.path.join(data_dir,data_file_name))
data_df = df.copy()
data_df = data_df.rename(columns={'item':'item_id'})

In [3]:
data_df = data_df.set_index('date')
data_df.store = data_df.store.astype(str)
data_df.item_id = data_df.item_id.astype(str)
data_df.index = pd.to_datetime(data_df.index, format = '%Y-%m-%d')
cols_order = ['item_id', 'store','sales']
data_df = data_df[cols_order]

data_df.head()

,item_id,store,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


In [4]:
data_df.tail()

,item_id,store,sales
date,,,
2017-12-27,50,10,63
2017-12-28,50,10,59
2017-12-29,50,10,74
2017-12-30,50,10,62
2017-12-31,50,10,82


In [5]:
print(data_df.index.name, data_df.index.dtype)
print(data_df.dtypes)

date datetime64[ns]
item_id    object
store      object
sales       int64
dtype: object


## 사용할 데이터 기간 선택 

여기서는 추후 Predictor 및 Forecast를 에 사용할 데이터 기간을 정합니다. 현재 5년의 데이타를 다 사용하게 되면 훈련 시간이 오래 걸려서, 여기서는 최근 2년의 데이터만을 사용 합니다.
- 훈련 데이터 셋 (Train Data Set)
    - 데이타의 시작날짜, 미지막 날짜를 확인하고, 실제 학습에 사용할 사용할 기간을 2015-01-01 부터 2017-12-01 까지 23개월을 사용
- 검증 데어터 셋 (Validation Data Set)
    - 사용할 기간을 2017-12-01-2017-12-31일까지 1달을 사용.

In [6]:
print(data_df.index.max())
print(data_df.index.min())
start_train_date = '2015-01-01' # Non-inclusive
end_train_date = '2017-12-01' # Non-inclusive
end_val_date = '2018-01-01' # Non-inclusive
stores_sales = data_df.copy()

2017-12-31 00:00:00
2013-01-01 00:00:00


In [7]:
train_stores_sales = stores_sales[stores_sales.index >= start_train_date]
train_stores_sales = train_stores_sales[train_stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]


In [8]:
train_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532500 entries, 2015-01-01 to 2017-11-30
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   item_id  532500 non-null  object
 1   store    532500 non-null  object
 2   sales    532500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 16.3+ MB


In [9]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15500 entries, 2017-12-01 to 2017-12-31
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  15500 non-null  object
 1   store    15500 non-null  object
 2   sales    15500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 484.4+ KB


## Train 및 Validation Data set을 CSV 파일로 저장

두 개의 데이타 셋을 두 개의 CSV 파일로 저장하여, 추후에 Forecast에서 Data Import시에 사용 합니다.

In [10]:
train_time_series_filename = "train_target_time_series.csv"
train_time_series_path = data_dir + "/" + train_time_series_filename
train_stores_sales.to_csv(train_time_series_path, header=False)

In [11]:
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

다음 노트북에서 사용을 하기 위해서, 이 노트북에서 저장한 변수를 디스크에 저장을 합니다.
만약 아래의 코드에 에러가 발생을 했다고 하면, 이 노트북을 처음부터 다시 실행 해주세요.

In [12]:
%store train_time_series_filename
%store train_time_series_path
%store train_stores_sales
%store validation_time_series_filename
%store validation_time_series_path
%store validation_stores_sales

Stored 'train_time_series_filename' (str)
Stored 'train_time_series_path' (str)
Stored 'train_stores_sales' (DataFrame)
Stored 'validation_time_series_filename' (str)
Stored 'validation_time_series_path' (str)
Stored 'validation_stores_sales' (DataFrame)


In [13]:
%store

Stored variables and their in-db values:
train_stores_sales                          ->            item_id store  sales
date              
train_time_series_filename                  -> 'train_target_time_series.csv'
train_time_series_path                      -> 'data/train_target_time_series.csv'
validation_stores_sales                     ->            item_id store  sales
date              
validation_time_series_filename             -> 'validation_time_series.csv'
validation_time_series_path                 -> 'data/validation_time_series.csv'
